
# 🧪 04 — RAG Hands-On (Python, Option A)

Welcome to the **hands-on lab** for building a full **Retrieval-Augmented Generation (RAG)** pipeline in Python.

In this notebook you will:

- Load **real sample documents** (finance, health, legal, code)
- Chunk them using several strategies
- Embed them with different embedding backends (OpenAI + local)
- Store vectors in a simple local vector store (Chroma or FAISS)
- Retrieve and re-rank chunks
- Generate answers with citations
- Add basic conversational memory
- Add simple evaluation hooks (LLM-as-judge style)

> ⚠️ You can run this notebook “locally-first”: all sample docs are contained in this repo under `data/sample_docs/`.



## 0. Environment Setup

Uncomment and run the following cells in your own environment.

```bash
# Create and activate a virtual env (optional)
python -m venv .venv
source .venv/bin/activate  # on Windows: .venv\Scripts\activate

# Install core dependencies
pip install "langchain>=0.3.0" "langchain-community" "langchain-openai"             "chromadb" "sentence-transformers" "pypdf" "python-dotenv"
```

Create a `.env` file in the repo root:

```bash
OPENAI_API_KEY=sk-...
```



## 1. Inspect Sample Documents

We included a few small sample documents under:

- `data/sample_docs/finance_intro.md`
- `data/sample_docs/health_intro.md`
- `data/sample_docs/legal_clause.md`
- `data/sample_docs/code_sample.py`


In [ ]:
from pathlib import Path

base_dir = Path("..").resolve() if (Path(".") / "notebooks").exists() else Path(".").resolve()
samples_dir = base_dir / "data" / "sample_docs"

list(samples_dir.glob("*"))

In [ ]:
for path in samples_dir.glob("*"):
    print(f"--- {path.name} ---")
    print(path.read_text()[:300], "...
")


## 2. Loading Documents

We'll keep loading logic simple and transparent:

- read `.md` and `.py` files directly
- you can later plug in PDF loaders such as `PyPDFLoader` or `PyMuPDFLoader`.


In [ ]:
from typing import List
from dataclasses import dataclass

@dataclass
class RawDocument:
    content: str
    metadata: dict

def load_markdown_files(folder: Path) -> List[RawDocument]:
    docs: List[RawDocument] = []
    for path in folder.glob("*.md"):
        text = path.read_text(encoding="utf-8")
        docs.append(RawDocument(content=text, metadata={"source": str(path)}))
    return docs

def load_code_files(folder: Path) -> List[RawDocument]:
    docs: List[RawDocument] = []
    for path in folder.glob("*.py"):
        text = path.read_text(encoding="utf-8")
        docs.append(RawDocument(content=text, metadata={"source": str(path)}))
    return docs

raw_docs = load_markdown_files(samples_dir) + load_code_files(samples_dir)
len(raw_docs), raw_docs[0]


## 3. Chunking Strategies

We’ll use LangChain's `RecursiveCharacterTextSplitter` for a robust baseline,
and show how to adjust chunk size and overlap.

Later, you can experiment with token-based or sentence-based splitters.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import Dict

@dataclass
class Chunk:
    content: str
    metadata: Dict

def chunk_documents(
    docs: List[RawDocument],
    chunk_size: int = 500,
    chunk_overlap: int = 100,
) -> List[Chunk]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " ", ""],
    )
    chunks: List[Chunk] = []
    for d in docs:
        for c in splitter.split_text(d.content):
            chunks.append(Chunk(content=c, metadata=d.metadata.copy()))
    return chunks

chunks = chunk_documents(raw_docs, chunk_size=400, chunk_overlap=80)
len(chunks), chunks[0]


> ✅ At this point you have a set of **chunks** with text and metadata (`source`).

You can inspect a few examples:


In [ ]:
for i, ch in enumerate(chunks[:5]):
    print(f"Chunk {i} from {ch.metadata['source']}:")
    print(ch.content)
    print("-" * 80)


## 4. Embeddings (OpenAI + Local)

We'll define a simple embedding interface and support multiple backends:

- OpenAI embeddings (e.g., `text-embedding-3-large`)
- Local `sentence-transformers` model (offline-friendly)

> ⚠️ Only run OpenAI code if you have an API key configured.


In [ ]:
from typing import Literal, Optional
import os

EmbedModel = Literal["openai", "local"]

class Embedder:
    def __init__(self, model: EmbedModel = "openai"):
        self.model_type = model
        self._local_model = None
        self._openai_client = None

    def _ensure_local_model(self):
        if self._local_model is None:
            from sentence_transformers import SentenceTransformer
            self._local_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    def _ensure_openai_client(self):
        if self._openai_client is None:
            from openai import OpenAI
            self._openai_client = OpenAI()

    def embed_texts(self, texts: list[str]) -> list[list[float]]:
        if self.model_type == "local":
            self._ensure_local_model()
            return self._local_model.encode(texts, convert_to_numpy=False).tolist()
        elif self.model_type == "openai":
            self._ensure_openai_client()
            resp = self._openai_client.embeddings.create(
                model="text-embedding-3-large",
                input=texts,
            )
            return [d.embedding for d in resp.data]
        else:
            raise ValueError(f"Unknown model_type={self.model_type!r}")



### 4.1 Embed Our Chunks (Local Model by Default)

We'll default to the local model so you can run without paying for tokens.


In [ ]:
embedder = Embedder(model="local")  # change to "openai" if you prefer and have key
texts = [c.content for c in chunks]
embeddings = embedder.embed_texts(texts)
len(embeddings), len(embeddings[0])


## 5. Simple Vector Store (Chroma)

For a hands-on lab, **Chroma** is a great local vector store.

We'll build a small wrapper around it to keep the interface simple.


In [ ]:
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document

class ChromaVectorStore:
    def __init__(self, persist_directory: Optional[str] = None):
        self.persist_directory = persist_directory
        self._store = None

    def build(self, chunks: list[Chunk]):
        from langchain_openai import OpenAIEmbeddings
        lc_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

        docs = [
            Document(page_content=c.content, metadata=c.metadata)
            for c in chunks
        ]

        self._store = Chroma.from_documents(
            documents=docs,
            embedding=lc_embeddings,
            persist_directory=self.persist_directory,
            collection_name="rag_demo",
        )

    def similarity_search(self, query: str, k: int = 5) -> list[Document]:
        return self._store.similarity_search(query, k=k)



### 5.1 Build the Vector Store

This will create (or reuse) a local Chroma directory.


In [ ]:
vs = ChromaVectorStore(persist_directory="chroma_rag_demo")
vs.build(chunks)


## 6. Retrieval + Answer Generation

We can now define a simple **RAG chain**:

1. Retrieve top-k documents from Chroma  
2. Compose a prompt with:
   - user question
   - retrieved context  
3. Call an LLM (OpenAI or local wrapper) to generate an answer with citations.


In [ ]:
from langchain_openai import ChatOpenAI

def build_context(docs, max_chars: int = 2000) -> str:
    ctx_parts = []
    total = 0
    for d in docs:
        snippet = d.page_content[:500]
        src = d.metadata.get("source", "unknown")
        block = f"Source: {src}\n{snippet}"
        if total + len(block) > max_chars:
            break
        ctx_parts.append(block)
        total += len(block)
    return "\n\n---\n\n".join(ctx_parts)

def rag_answer(query: str, k: int = 5, model_name: str = "gpt-4o-mini") -> str:
    docs = vs.similarity_search(query, k=k)
    context = build_context(docs)
    system_prompt = (
        "You are a careful assistant. Use ONLY the provided context to answer.\n"
        "If the context is insufficient, say you are not sure. Always include brief citations."
    )
    llm = ChatOpenAI(model=model_name)
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Question: {query}\n\nContext:\n{context}"},
    ]
    resp = llm.invoke(messages)
    return resp.content


### 6.1 Try a Demo Question

Once your environment is set up and you have an OpenAI key, you can run:


In [ ]:
# Example (requires OPENAI_API_KEY)
# answer = rag_answer("Explain dollar-cost averaging in simple terms.")
# print(answer)


## 7. Conversational RAG (Simple Memory)

We can add a very light-weight **conversation buffer**:

- Keep the last few Q&A pairs
- Include them in the prompt as extra context (but separate from retrieved docs)


In [ ]:
from collections import deque

class ConversationRAG:
    def __init__(self, history_limit: int = 5, model_name: str = "gpt-4o-mini"):
        self.history = deque(maxlen=history_limit)
        self.model_name = model_name

    def ask(self, query: str, k: int = 5) -> str:
        docs = vs.similarity_search(query, k=k)
        rag_context = build_context(docs)

        history_str = ""
        if self.history:
            history_str = "\n\nConversation history:\n"
            for i, (q, a) in enumerate(self.history):
                history_str += f"Q{i+1}: {q}\nA{i+1}: {a}\n"

        system_prompt = (
            "You are a conversational RAG assistant.\n"
            "Use the retrieved context and (secondarily) the conversation history.\n"
            "Be honest if you do not know."
        )
        llm = ChatOpenAI(model=self.model_name)
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"{history_str}\n\nNew question: {query}\n\nRetrieved context:\n{rag_context}"},
        ]
        resp = llm.invoke(messages)
        answer = resp.content
        self.history.append((query, answer))
        return answer

conv_rag = ConversationRAG()



You could then run, for example:

```python
# conv_rag.ask("Explain dollar-cost averaging.")
# conv_rag.ask("How is that different from investing a lump sum?")
```



## 8. Simple Evaluation Hook (LLM-as-Judge Concept)

We can ask an LLM to self-evaluate RAG answers:

- Given: question, context, answer
- Ask: how faithful and useful is this?

> ⚠️ This is a light, conceptual version of what you’ll see more formally
> in `07_RAG_Evaluation_and_Benchmarks.ipynb`.


In [ ]:
def judge_answer(question: str, answer: str, context: str, model_name: str = "gpt-4o-mini") -> str:
    prompt = f"""You are evaluating a Retrieval-Augmented Generation (RAG) answer.

Question:
{question}

Context:
{context}

Answer:
{answer}

Evaluate:
- Faithfulness to the context (0–10)
- Helpfulness (0–10)
- Brief explanation of any issues.

Return a short evaluation report.
"""
    llm = ChatOpenAI(model=model_name)
    resp = llm.invoke([{"role": "user", "content": prompt}])
    return resp.content

# Example usage (once you have an answer and context):
# docs = vs.similarity_search("Explain dollar-cost averaging", k=4)
# ctx = build_context(docs)
# ans = rag_answer("Explain dollar-cost averaging", k=4)
# print(judge_answer("Explain dollar-cost averaging", ans, ctx))



## 9. Next Steps

From here, you can:

- Swap in **FAISS** instead of Chroma
- Add **BM25** for hybrid retrieval
- Integrate with your **Agents Universe** (Agentic RAG)
- Wrap this into a **FastAPI** or **Streamlit** app
- Use the evaluation ideas from `07_RAG_Evaluation_and_Benchmarks.ipynb` to compare:
  - different chunking strategies
  - different embedding models
  - different retrieval parameters

This notebook is your **entry point** into the Python side of your RAG Universe — a bridge between:

- the conceptual reports & diagrams, and  
- real, executable RAG pipelines.
